# LangChain Expression Language

Explain the basic syntax of LangChain Expression Language, which uses the pipe symbol | to connect components. Each component represents a specific task or action.

To make it as easy as possible to create custom chains, LangChain implemented a Runnable protocol. The Runnable protocol is implemented for most components. This is a standard interface, which makes it easy to define custom chains as well as invoke them in a standard way. The standard interface includes:

* stream: stream back chunks of the response
* invoke: call the chain on an input
* batch: call the chain on a list of inputs

These also have corresponding async methods:

* astream: stream back chunks of the response async
* ainvoke: call the chain on an input async
* abatch: call the chain on a list of inputs async
* astream_log: stream back intermediate steps as they happen, in addition to the final response


# The Runnable Protocol:
A unit of work that can be invoked, batched, streamed, transformed and composed.

All methods accept an optional config argument, which can be used to configure execution, add tags and metadata for tracing and debugging etc.

Runnables expose schematic information about their input, output and config via the input_schema property, the output_schema property and config_schema method.

The LangChain Expression Language (LCEL) is a declarative way to compose Runnables into chains. Any chain constructed this way will automatically have sync, async, batch, and streaming support.

The main composition primitives are RunnableSequence and RunnableParallel.



In [1]:
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv


In [2]:
print(type(RunnableLambda(lambda x: x + 1)))

<class 'langchain_core.runnables.base.RunnableLambda'>


In [4]:
chain = RunnableLambda(lambda x: x + 1)

In [5]:
print(chain.invoke(1))
print(chain.invoke(2))

2
3


# RunnableSequence
RunnableSequence invokes a series of runnables sequentially, with one runnable’s output serving as the next’s input. Construct using the | operator or by passing a list of runnables to RunnableSequence.

In [7]:
sequence = RunnableLambda(lambda x: x + 1) | (lambda x: x + 2)

print(type(sequence))
print(sequence.invoke(1))
sequence.batch([1, 2, 3])

<class 'langchain_core.runnables.base.RunnableSequence'>
4


[4, 5, 6]

# Runnable Parallel
The `RunnableParallel`, allows for multiple runnables to be invoked in parallel, construct using a dictionary of runnables to invoke in parallel.

In [8]:
sequence = RunnableLambda(lambda x: x + 1) | {
    "mul_2": RunnableLambda(lambda x: x * 2),
    "mul_5": RunnableLambda(lambda x: x * 5),
}
sequence.invoke(1)

{'mul_2': 4, 'mul_5': 10}

# Combining functions with parallel

In [9]:
sequence = RunnableLambda(lambda x : x + 1) | {
    'mul-2': RunnableLambda(lambda x : x * 2),
    'mul-5': RunnableLambda(lambda x: x * 5) 
} | RunnableLambda(lambda x : x['mul-2'] + x['mul-5'])

sequence.invoke(1)

14

In [13]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel({
    'mul-2': RunnableLambda(lambda x: x * 2),
    'mul-5': RunnableLambda(lambda x: x * 5)
})

# This is a dictionary, however it will be composed with other runnables when used in a sequence:
parallel_two = {
    'mul_2': RunnableLambda(lambda x: x['input_one'] * 2),
    'mul_5': RunnableLambda(lambda x: x['input_two'] * 5)
}

print(type(parallel)) # <class 'langchain.schema.runnable.RunnableParallel'>
print(type(parallel_two)) # <class 'dict'>

<class 'langchain_core.runnables.base.RunnableParallel'>
<class 'dict'>


In [15]:

chain = parallel | RunnableLambda(lambda x: x['mul-2'] + x['mul-5']) 
chain.invoke(5)

35

In [16]:
second_chain = parallel_two | RunnableLambda(lambda x: x['mul_2'] + x['mul_5']) 
second_chain.invoke({'input_one': 5, 'input_two': 10})

60

In [17]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel({
    'item_one': RunnableLambda(lambda x: f"Hello {x['name']} "),
    'item_two': RunnableLambda(lambda x: 'Welcome to the World!')
})

In [18]:
def combine(x):
    return x['item_one'] + x['item_two']

In [ ]:
parallel_chain_example = parallel | combine
parallel_chain_example.invoke({'name': "Pranav"}) 

'Hello Pranav Welcome to the World!'

In [ ]:
lambda_example = RunnableLambda(lambda x: {'item_one': 'Hello ', 'item_two': 'World'})
lambda_chain_example = lambda_example | combine
lambda_chain_example.invoke({})

'Hello World'